In [ ]:
!pip install open3d

In [3]:
from torchvision.transforms import Compose, ToTensor
import torch

import os
import random
from random import sample

import numpy as np
import pandas as pd

from torch.utils.data import Dataset, DataLoader
import open3d as o3d
# to make more deterministic the task
seed = 0
random.seed(seed)
torch.manual_seed(seed)
np.random.seed(seed)

In [76]:
### PARAMS
INPUT_EXTENTION = ".ply"
ROOT_FOLDER = os.path.dirname("/content/drive/MyDrive/Output_ROTATED_v7/")

In [ ]:
import tensorflow as tf
device_name = tf.test.gpu_device_name()
if device_name != '/device:GPU:0':
  raise SystemError('GPU device not found')
print('Found GPU at: {}'.format(device_name))

Found GPU at: /device:GPU:0


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
#!unzip '/content/drive/MyDrive/Output_ROTATED_v7.zip' -d "/content/drive/MyDrive/"

In [119]:
folders = [dir for dir in sorted(os.listdir(ROOT_FOLDER)) if os.path.isdir(os.path.join(ROOT_FOLDER,dir))]
print(folders)
classes = {folder:i for i, folder in enumerate(folders)};
classes

['bathtub', 'bed', 'chair', 'desk', 'dresser', 'monitor', 'night_stand', 'sofa', 'table', 'toilet']


{'bathtub': 0,
 'bed': 1,
 'chair': 2,
 'desk': 3,
 'dresser': 4,
 'monitor': 5,
 'night_stand': 6,
 'sofa': 7,
 'table': 8,
 'toilet': 9}

In [83]:
def read_voxelGrid(filename):
    if os.path.isfile(filename) and os.path.join(filename).endswith(INPUT_EXTENTION):
      voxelGrid = o3d.io.read_voxel_grid(filename, format='auto', print_progress=True)
      np_voxels = np.array(list(map(lambda x: x.grid_index, voxelGrid.get_voxels())))
      #print(len(np_voxels))

      np_voxelGrid = np.zeros((32, 32, 32))
      for j in range(len(np_voxels)):
        x = np_voxels[j][0]
        y = np_voxels[j][1]
        z = np_voxels[j][2]
        np_voxelGrid[x,y,x] = 1
        #print(np_voxelGrid)
      
      #print(np_voxelGrid.shape)
      return np_voxelGrid
    else: raise('Not a valid PLY file, is the location correct?')

In [225]:
class VoxelGridDataset(Dataset):
    def __init__(self, dataset_folder, folder="train", transform=None):
        self.dataset_folder = dataset_folder
        folders = [dir for dir in sorted(os.listdir(dataset_folder)) if os.path.isdir(dataset_folder+"/"+dir)]
        self.classes = {folder: i for i, folder in enumerate(folders)}
        self.transforms = transform
        self.files = []
        for category in classes.keys():
          model_folder = os.path.join(dataset_folder,category,folder)
          for file in os.listdir(model_folder):
            if file.endswith(INPUT_EXTENTION):
              sample = {}
              sample['path'] = os.path.join(model_folder,file)
              sample['category'] = category
              self.files.append(sample)

    def __len__(self):
        return len(self.files)

    def __preproc__(self, file):
        voxelGrid = read_voxelGrid(file)
        if self.transforms:
            voxelGrid = self.transforms(voxelGrid)

        return voxelGrid

    def __getitem__(self, idx):
        path = self.files[idx]['path']
        #print(path)
        category = self.files[idx]['category']
        voxelGrid = self.__preproc__(path)
        cat = [0 for _ in range(10)]
        cat[self.classes[category]] = 1
        x_cat = torch.tensor(cat)
        # return {'voxelGrid': voxelGrid, 'category': tuple(cat)}
        return voxelGrid, x_cat

In [218]:
dataset_folder = ROOT_FOLDER
folders = [dir for dir in sorted(os.listdir(dataset_folder)) if os.path.isdir(dataset_folder+"/"+dir)]
print(folders)
folder = "train"
files = []
for category in classes.keys():
  model_folder = os.path.join(dataset_folder,category,folder)
  for file in os.listdir(model_folder):
    if file.endswith(INPUT_EXTENTION):
      #print(model_folder+"/"+file)
      files.append(file)
print(len(files))

['bathtub', 'bed', 'chair', 'desk', 'dresser', 'monitor', 'night_stand', 'sofa', 'table', 'toilet']
9986


In [226]:
transforms = Compose([
    ToTensor(), #this converts numpy or image to torch tensor and normalizes it in 0, 1
])

train_dataset =  VoxelGridDataset(ROOT_FOLDER, "train", transforms)
test_dataset =  VoxelGridDataset(ROOT_FOLDER, "test", ToTensor())

#valid_dataset =  VoxelGridDataset(ROOT_FOLDER, "valid", ToTensor())

In [227]:
print('Train dataset size: ', len(train_dataset))
print('Test dataset size: ', len(test_dataset))
print('Number of classes: ', len(train_dataset.classes))
print('Sample VoxelGrid shape: ', train_dataset)
print('Class: ', train_dataset[0])
# print('Class: ', train_dataset[1000]['category'])
# print('Class: ', train_dataset[8000]['category'])
# print('Class: ', train_dataset[8900]['category'])
# print('Class: ', train_dataset[9900]['category'])
# folders[train_dataset[9900]['category'].index(1)]

Train dataset size:  9986
Test dataset size:  2724
Number of classes:  10
Sample VoxelGrid shape:  <__main__.VoxelGridDataset object at 0x7f8d62915a60>
Class:  (tensor([[[1., 1., 1.,  ..., 1., 1., 1.],
         [0., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 0.,  ..., 0., 0., 0.],
         ...,
         [0., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 0.,  ..., 0., 0., 0.]],

        [[0., 0., 0.,  ..., 0., 0., 0.],
         [1., 1., 1.,  ..., 1., 1., 1.],
         [0., 0., 0.,  ..., 0., 0., 0.],
         ...,
         [0., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 0.,  ..., 0., 0., 0.]],

        [[0., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 0.,  ..., 0., 0., 0.],
         [1., 1., 1.,  ..., 1., 1., 1.],
         ...,
         [0., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 0.,  ..., 0., 0., 0.]],

        ...,

        [[0., 0., 0.,  ..., 0., 0., 0.]

In [228]:
batch_size = 64
nw=os.cpu_count()
train_dataloader = DataLoader(train_dataset, batch_size, shuffle=True, num_workers=nw)
test_dataloader = DataLoader(test_dataset, batch_size, shuffle=False, num_workers=nw)

#valid_dataloader = DataLoader(valid_dataset, batch_size, shuffle=False, num_workers=nw)

In [ ]:
def getDimCNN(W,K,P,S):
  """[(W−K+2P)/S]+1.
    W is the input volume
    K is the Kernel size
    P is the padding
    S is the stride"""
  n = ((W-K+2*P)/S)+1
  return n

In [ ]:
print(getDimCNN(32,3,1,2))

16.5


In [ ]:
def cutout_3d(volume, cutout_size=(8,8,8)):
    # Shape is 32x32x32
    x = np.random.randint(0, 32 - cutout_size[0])
    y = np.random.randint(0, 32 - cutout_size[1])
    z = np.random.randint(0, 32 - cutout_size[2])
    cutout_cube = (x, y, z, cutout_size[0], cutout_size[1], cutout_size[2])
    volume[x:x+cutout_size[0], y:y+cutout_size[1], z:z+cutout_size[2]] = 0
    return volume

In [ ]:
from keras.utils import to_categorical

X_validation, y_validation = prepare_data(test[1], "test")
y_validation = to_categorical(y_validation, 10) 

In [ ]:
idt = np.random.permutation(len(X_validation))
X_validation, y_validation = X_validation[idt], y_validation[idt]

# HERE

In [159]:
from torch.nn import Module, Sequential, Conv2d, BatchNorm2d, ReLU

In [237]:
#from torch.nn.modules.dropout import Dropout
from torch.nn import Module, Linear, Sequential, Conv3d, BatchNorm3d, ReLU, MaxPool3d, Dropout, Dropout3d

class VGNet(Module):
    def __init__(self):
        super().__init__()
        kernel_size = 3
        padding_val = kernel_size // 2

        self.network = Sequential(
            # OPT B: -> CONV/FC -> ReLu(or other activation) -> Dropout -> BatchNorm -> CONV/FC
            # Block 1 : > CONV -> BatchNorm -> ReLu -> Dropout ->
            Conv3d(1, 16, kernel_size=3, stride=1, padding=padding_val), #32-->16 (if s=2)
            BatchNorm3d(16),
            ReLU(),
            Dropout(0.2),
            MaxPool3d(kernel_size=3, stride=2), # H/3 -->

            # Block 2 : > CONV -> BatchNorm -> ReLu -> Dropout ->
            Conv3d(16, 32, kernel_size=3, stride=1, padding=padding_val), #32-->16 (if s=2)
            BatchNorm3d(32),
            ReLU(),
            Dropout(0.2),
            MaxPool3d(kernel_size=3, stride=2), # H/3

            # Block 3 : > CONV -> BatchNorm -> ReLu -> Dropout ->
            Conv3d(32, 64, kernel_size=3, stride=2, padding=padding_val), #32-->16 (if s=2)
            BatchNorm3d(64),
            ReLU(),
            Dropout(0.2),
            MaxPool3d(kernel_size=3, stride=2), # H/3
        )
        self.classification_layer = Linear(64, 10) #1124864 | 1728
        self.apply(self._init_weights)

    def _init_weights(self, module):
        if isinstance(module, torch.nn.Linear):
            torch.nn.init.xavier_uniform_(module.weight)
            if module.bias is not None:
                module.bias.data.zero_()
        if isinstance(module, torch.nn.Conv3d):
            torch.nn.init.xavier_uniform_(module.weight)
            if module.bias is not None:
                module.bias.data.zero_()

    def forward(self, x):
        #print((x.shape))
        y = self.network(x).reshape((x.shape[0], -1))
        #print(y.shape)
        y = self.classification_layer(y)
        return y

In [161]:
device = "cuda" if torch.cuda.is_available() else "cpu"
device

'cpu'

In [238]:
from torchsummary import summary
network = VGNet()
network.to(device)
# print(np.random.rand(1,3,5,5))
# summary(network, (1,32,32,32))
# print(network)
temp = torch.rand(1,1,32,32,32)
print(network(temp))

tensor([[ 4.5546,  1.2692,  0.1168,  3.6142,  2.3000,  2.5117, -1.1824,  1.0298,
          1.8317,  0.0252]], grad_fn=<AddmmBackward0>)


In [236]:
from torch.optim import SGD, Adam
from torch.nn import BCEWithLogitsLoss
from tqdm import tqdm
model = VGNet()
opt = SGD(model.parameters(), lr=1e-2, weight_decay = 0)
loss_fn = BCEWithLogitsLoss()
device = "cuda" if torch.cuda.is_available() else "cpu"
model.to(device)
epochs=10
best_val = np.inf
for epoch in range(epochs):
    model.train()
    print(f"Epoch: {epoch+1}")
    iterator = tqdm(train_dataloader)
    for batch_x, batch_y in iterator:
        #print(batch_x,batch_y)
        # batch_x = batch_x[None, :]
        batch_x = torch.unsqueeze(batch_x, dim=1)
        batch_x = batch_x.float()

        batch_x = batch_x.to(device)
        batch_y = batch_y.to(device)

        y_pred = model(batch_x)

        loss = loss_fn(y_pred, batch_y.float())

        opt.zero_grad()
        loss.backward()
        opt.step()
        iterator.set_description(f"Train loss: {loss.detach().cpu().numpy()}")

    model.eval()
    with torch.no_grad():
        predictions = []
        true = []
        for batch_x, batch_y in tqdm(valid_dataloader):
            
            batch_x.reshape(1,...)
            batch_x = batch_x.float()
            batch_y = batch_y.float()
            batch_x = batch_x.to(device)
            batch_y = batch_y.to(device)

            y_pred = model(batch_x)

            predictions.append(y_pred)
            true.append(batch_y)
        predictions = torch.cat(predictions, axis=0)
        true = torch.cat(true, axis=0)
        val_loss = loss_fn(predictions, true)
        val_acc = (torch.sigmoid(predictions).round() == true).float().mean()
        print(f"loss: {val_loss}, accuracy: {val_acc}")
    
    if val_loss < best_val:
        print("Saved Model")
        torch.save(model.state_dict(), "model.pt")
        best_val = val_loss
        


Epoch: 1


  0%|          | 0/157 [00:00<?, ?it/s]

torch.Size([64, 1, 32, 32, 32])
torch.Size([64, 64])


Train loss: 1.9977413415908813:   1%|          | 1/157 [00:08<22:21,  8.60s/it]

torch.Size([64, 1, 32, 32, 32])
torch.Size([64, 64])


...

torch.Size([64, 1, 32, 32, 32])
torch.Size([64, 64])


Train loss: 0.4601217806339264:   3%|▎         | 5/157 [00:20<09:02,  3.57s/it]

torch.Size([64, 1, 32, 32, 32])
torch.Size([64, 64])


Train loss: 0.4601217806339264:   3%|▎         | 5/157 [00:23<12:07,  4.79s/it]


KeyboardInterrupt: ignored

In [ ]:
from torch.optim import SGD, Adam
from torch.nn import BCEWithLogitsLoss, CrossEntropyLoss
from tqdm import tqdm

TOT_MODELS = 2

Histories = []
model_ = np.empty(TOT_MODELS, dtype=object)
#Repeat 5 times
for i in range(TOT_MODELS):
  #Compile the model
  model_[i] = VGNet()
  criterion = CrossEntropyLoss()
  optimizer = SGD(model_[i].parameters(), lr=0.01)
  patience = 10
  train_losses = []
  val_losses = []
  best_acc = 0.0

  for epoch in range(30):
      # Train
      model_[i].train()
      train_loss = 0.0
      #X_train_tensor = torch.from_numpy(X_train).float()

      for inputs, labels in zip(X_train_tensor, y_train_tensor):

          optimizer.zero_grad()
          # print(inputs.shape)
          # inputs.reshape(1, 1, 32, 32, 32)
          # print(inputs.shape)
          outputs = model_[i](inputs)
          loss = criterion(outputs, labels)
          loss.backward()
          optimizer.step()
          train_loss += loss.item()
      train_losses.append(train_loss / len(X_train))

      # Validate
      model_[i].eval()
      val_loss = 0.0
      correct = 0
      total = 0
      with torch.no_grad():
          for inputs, labels in zip(X_validation, y_validation):
              inputs = inputs.to(device)
              labels = labels.to(device)

              outputs = model_[i](inputs)
              print("OUTPUTS" + str(outputs.shape))
              loss = criterion(outputs, labels)
              val_loss += loss.item()
              _, predicted = torch.max(outputs.data, 1)
              total += labels.size(0)
              correct += (predicted == labels).sum().item()
      val_loss /= len(X_validation)
      val_losses.append(val_loss)
      acc = 100. * correct / total
      if acc > best_acc:
          counter = 0
          best_acc = acc
          torch.save(model_[i].state_dict(), 'best_model.pth')
          #early_stop.best = acc
      else:
        counter = counter + 1
      if counter >= patience:
          print("Early stopping")
          break

      print("Early Stop called or End of Epochs reaches: Saving the model")
      #Save the model
      model_name = "Model" + str(i) + ".pth"
      os.rename("best_model.h5", model_name)
      #Histories.append(history)

torch.Size([1, 32, 32, 32])
torch.Size([1, 32, 32, 32])
torch.Size([1, 32, 32, 32])


ValueError: ignored

In [ ]:
for i in range(5):
  score = model_[i].evaluate(X_validation,y_validation)
  print("The score of Model " + str(i) + " is")
  print(score[1])

In [ ]:
import matplotlib.pyplot as plt

#plot the train/validation loss and accuracy
def plot_graphs(history, metric):
    
    plt.plot(history.history[metric])
    plt.plot(history.history['val_'+metric], '')
    plt.xlabel("Epochs")
    plt.ylabel(metric)
    plt.legend([metric, 'val_'+metric])


plt.figure(figsize=(16, 6))
plt.subplot(1, 2, 1)
plot_graphs(Histories[0], 'accuracy')
plt.subplot(1, 2, 2)
plot_graphs(Histories[0], 'loss')